In [ ]:
##### Jeu de données #####

# Import des données
import pandas as pd
pam50 = pd.read_csv("Data_Pam50.csv")

# Extraction des features (niveau d'expression de divers gènes) et des labels (type de cancer)
features = pam50[pam50.columns[2:]]
labels = pam50["subtype"]
print("features :", list(features.columns))
print(len(list(features.columns)))




In [101]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

#Paramétrage du classifieur
classifier = tree.DecisionTreeClassifier(max_depth = 3)


score = 0
i= 0
while(i < 1000) :
    # Séparation des jeux de données et test
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.1)
    while(set(train_labels.unique()) != set(test_labels.unique())) :
        train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.1)
    
    # Apprentissage
    classifier = classifier.fit(train_features, train_labels)

    # Test
    score += metrics.accuracy_score(test_labels,classifier.predict(test_features))
    s +=1
    i +=1
    
print(round(score/10))



77


In [ ]:
"""

Stopping criterias :

max_depth = None : 75%
max_depth = 5 : 75%
max_debth = 4 : 76%
max_depth = 3 : 78%
max_depth = 2 : 57%
max_depth = 1 : 50%
_____________________________
Approach :

criterion = "gini" : 75%
criterion = "entropy" : 73%

splitter = "best" : 75%
splitter = "random" : 75%

max_features = 50/None : 75%
max_features = 25 : 73%
max_features = 10 : 71%
max_features = 5 : 69%
max_features = 1 :60
_____________________________
Other :
ccp_alpha = 0 : 75%
ccp_alpha = 0.001 : 76%
ccp_alpha = 0.005 : 76%
ccp_alpha = 0.01 : 76%
ccp_alpha = 0.1 : 63%
ccp_alpha = 1 : 25%

_____________________________
Combinaisons : 

ccp_alpha = 0.005, max_depth = 4 : 76
ccp_alpha = 0.005, max_depth = 3 : 77
ccp_alpha = 0.005, max_depth = 3 : 58

max_depth = 3, criterion="entropy" : 73
max_depth = 3, splitter = "random" : 68

"""